<a href="https://colab.research.google.com/github/anujavenkatachalam04/anujavenkatachalam04/blob/main/notebooks/extract_remote_sensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract monthly remote sensing data - NDWI, NDBI from sentinel 2 & LULC from Dynamic World

In [ ]:
# # import dependencies
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

In [1]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import uuid
import ee
from concurrent.futures import ThreadPoolExecutor, as_completed


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
os.chdir("/content/drive/MyDrive/CHVI")

In [12]:
#--- Initialize Earth Engine ---
ee.Authenticate()
ee.Initialize(project="friendly-plane-476109-q8")

In [3]:
# --- Paths ---
grid_path = "5_Shapefiles/Rajasthan_1deg.geojson"
output_folder = "1_Data/Remote_Sensing/Raw"
os.makedirs(output_folder, exist_ok=True)

In [4]:
# Import grid
grid = gpd.read_file(grid_path)

In [5]:
grid.crs, len(grid)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Define Sentinel-2 masking (by pixels)

In [6]:
def mask_s2_clouds(image):
    """
    Masks clouds and shadows using Sentinel-2 Scene Classification Layer (SCL).
    Removes:
    3 = cloud shadow
    8 = cloud (medium)
    9 = cloud (high)
    10 = cirrus
    11 = snow/ice
    """
    scl = image.select("SCL")
    mask = (scl.neq(3)
              .And(scl.neq(8))
              .And(scl.neq(9))
              .And(scl.neq(10))
              .And(scl.neq(11)))
    return image.updateMask(mask)


# NDVI Extraction
- Uses bands B8, B4

In [13]:
def compute_monthly_ndvi(start_date, geom):
    """
    Computes mean monthly NDVI at a point geometry.
    Includes:
        ✔ 1-month primary window
        ✔ ±30 day fallback if primary empty
        ✔ Cloud masking using SCL
        ✔ Safe handling when no images
    """
    start = ee.Date(start_date)
    end = start.advance(1, "month")

    # --- Primary window ---
    s2_primary = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start, end)
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 60))
        .map(mask_s2_clouds))

    # --- Fallback ±30 days ---
    s2_fallback = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start.advance(-30, "day"), end.advance(30, "day"))
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 60))
        .map(mask_s2_clouds))

    # Choose primary if available, else fallback
    s2 = ee.ImageCollection(
        ee.Algorithms.If(s2_primary.size().gt(0), s2_primary, s2_fallback)
    )

    # Safety check: fallback might also be empty
    s2_nonempty = ee.ImageCollection(s2).size().gt(0)

    # Compute NDVI only if nonempty
    def compute_value():
        ndvi_collection = s2.map(
            lambda img: img.normalizedDifference(["B8", "B4"]).rename("NDVI")
        )
        mosaic = ndvi_collection.mean()
        return mosaic.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=10  # B8 (NIR) and B4 (Red) = 10m
        ).get("NDVI")

    return ee.Algorithms.If(s2_nonempty, compute_value(), None)


# NDWI Extraction
- Uses Green (B3) & NIR (B8)

In [7]:
def compute_monthly_ndwi(start_date, geom):
    """
    Computes mean monthly NDWI at a point geometry.
    Includes:
        ✔ Primary 1-month window
        ✔ Fallback ±30 days if no images
        ✔ Cloud masking (pixel-level)
        ✔ Safe handling when fallback also has zero images
    """
    start = ee.Date(start_date)
    end = start.advance(1, "month")

    # --- Primary 1-month window ---
    s2_primary = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start, end)
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 60))
        .map(mask_s2_clouds))

    # --- Fallback window: ±30 days ---
    s2_fallback = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start.advance(-30, "day"), end.advance(30, "day"))
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 60))
        .map(mask_s2_clouds))

    # --- Choose collection (prefer primary) ---
    s2 = ee.ImageCollection(
        ee.Algorithms.If(s2_primary.size().gt(0), s2_primary, s2_fallback)
    )

    # If fallback ALSO empty → return None
    s2_nonempty = ee.ImageCollection(s2).size().gt(0)

    def compute_value():
        ndwi_collection = s2.map(
            lambda img: img.normalizedDifference(["B3", "B8"]).rename("NDWI")
        )
        mosaic = ndwi_collection.mean()
        return mosaic.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=10
        ).get("NDWI")

    # Using ee.Algorithms.If to avoid errors
    return ee.Algorithms.If(s2_nonempty, compute_value(), None)


# NDBI Extraction
- Uses SWIR (B11) & NIR (B8)

In [9]:
def compute_monthly_ndbi(start_date, geom):
    """
    Computes mean monthly NDBI at a point geometry.
    Follows:
        ✔ 1-month primary window
        ✔ ±30 day fallback if primary empty
        ✔ Cloud masking using SCL
        ✔ Safe return None if no imagery available
        ✔ Correct resolution for SWIR (B11 is 20m)
    """
    start = ee.Date(start_date)
    end = start.advance(1, "month")

    # --- Primary window ---
    s2_primary = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start, end)
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 60))
        .map(mask_s2_clouds))

    # --- Fallback window: ±30 days ---
    s2_fallback = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start.advance(-30, "day"), end.advance(30, "day"))
        .filterBounds(geom)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 60))
        .map(mask_s2_clouds))

    # Choose primary if available, else fallback
    s2 = ee.ImageCollection(
        ee.Algorithms.If(s2_primary.size().gt(0), s2_primary, s2_fallback)
    )

    # Safety check: fallback might also be empty
    s2_nonempty = ee.ImageCollection(s2).size().gt(0)

    # Compute NDBI only if nonempty
    def compute_value():
        ndbi_collection = s2.map(
            lambda img: img.normalizedDifference(["B11", "B8"]).rename("NDBI")
        )
        mosaic = ndbi_collection.mean()
        return mosaic.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=20  # SWIR band (B11) is 20m native resolution
        ).get("NDBI")

    # Prevent EE error on empty collections
    return ee.Algorithms.If(s2_nonempty, compute_value(), None)


# LULC

In [10]:
def compute_monthly_lulc(start_date, geom):
    """
    Computes the dominant Dynamic World land cover class for a point geometry.
    Includes:
        ✔ 1-month primary window
        ✔ ±30 day fallback if primary empty
        ✔ Mode of classes
        ✔ Safe handling for empty collections
        ✔ Works with point geometries
    """
    start = ee.Date(start_date)
    end = start.advance(1, "month")

    # --- Primary month ---
    dw_primary = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
        .filterDate(start, end)
        .filterBounds(geom))

    # --- Fallback ±30 days ---
    dw_fallback = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
        .filterDate(start.advance(-30, "day"), end.advance(30, "day"))
        .filterBounds(geom))

    # --- Choose primary if available, else fallback ---
    dw = ee.ImageCollection(
        ee.Algorithms.If(dw_primary.size().gt(0), dw_primary, dw_fallback)
    )

    # Safety check: if empty, return None
    dw_nonempty = dw.size().gt(0)

    def compute_mode():
        # Use mode of 'label' band to get dominant class
        mode_img = dw.select("label").reduce(ee.Reducer.mode())
        # Reduce at point
        return mode_img.reduceRegion(
            reducer=ee.Reducer.first(),  # first is sufficient at a point
            geometry=geom,
            scale=10
        ).get("mode")

    return ee.Algorithms.If(dw_nonempty, compute_mode(), None)


# Run code

In [11]:
# --- Date range ---
date_range = pd.date_range(start="2024-01-01", end="2025-10-31", freq="MS")

# --- Wrapper to run a function on a single row ---
def run_indicator(row, date, func, indicator_name):
    geom = ee.Geometry.Point(row.geometry.x, row.geometry.y)
    try:
        value = func(date, geom)
        # Force evaluation if using client-side
        if hasattr(value, 'getInfo'):
            value = value.getInfo()
    except Exception:
        value = None
    return {"grid_id": row["grid_id"], indicator_name: value}

# --- Generic processor ---
def process_monthly(grid, date, func, indicator_name, output_folder, max_workers=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(run_indicator, row, date, func, indicator_name): idx
            for idx, row in grid.iterrows()
        }
        for f in tqdm(as_completed(futures), total=len(futures),
                      desc=f"{indicator_name} {date.strftime('%Y-%m')}"):
            results.append(f.result())

    df = pd.DataFrame(results)
    df["Date"] = date.strftime("%Y-%m")
    output_path = f"{output_folder}/{indicator_name}_{date.strftime('%Y-%m')}.csv"
    df.to_csv(output_path, index=False)
    print(f"Saved {indicator_name} data for {date.strftime('%Y-%m')} → {output_path}")

# --- Main loop ---
for dt in date_range:
    # 1️⃣ NDWI
    process_monthly(grid, dt, compute_monthly_ndwi, "NDWI", output_folder)

    # # 2️⃣ NDBI
    # process_monthly(grid, dt, compute_monthly_ndbi, "NDBI", output_folder)

    # # 3️⃣ LULC
    # process_monthly(grid, dt, compute_monthly_lulc, "LULC", output_folder)


    # # 3️⃣ NDVI
    # process_monthly(grid, dt, compute_monthly_ndvi, "NDVI", output_folder)

NDWI 2024-01:   0%|          | 0/3095 [00:00<?, ?it/s]


EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

In [20]:
# Identify invalid or non-polygon geometries
invalid = grid[
    (~grid.is_valid) |
    (grid.geometry.is_empty) |
    (~grid.geometry.type.isin(["Polygon", "MultiPolygon"]))
]

print("Invalid geometries:")
print(invalid[['grid_id', 'geometry']])

Invalid geometries:
      grid_id                   geometry
0         111  POINT (69.58154 26.76008)
1         112  POINT (69.58154 26.86008)
2         113  POINT (69.58154 26.96008)
3         114  POINT (69.58154 27.06008)
4         115  POINT (69.58154 27.16008)
...       ...                        ...
3090     6243  POINT (77.98154 26.76008)
3091     6244  POINT (77.98154 26.86008)
3092     6316  POINT (78.08154 26.76008)
3093     6317  POINT (78.08154 26.86008)
3094     6390  POINT (78.18154 26.86008)

[3095 rows x 2 columns]
